How do we fit a function to data

In [4]:
from ipywidgets import interact
from fastai.basics import *

plt.rc('figure', dpi=90)

def plot_function(f, title=None, min=-2.1, max=2.1, color='r', ylim=None):
    x = torch.linspace(min,max, 100)[:,None]
    if ylim: plt.ylim(ylim)
    plt.plot(x, f(x), color)
    if title is not None: plt.title(title)

In [ ]:
def f(x): return 3*x**2 + 2*x + 1
plot_function(f, "$3x^2 + 2x + 1$")

In [10]:
def quad(a, b, c, x): return a*x**2 + b*x + c

In [11]:
quad(3,2,1,1.5)

10.75

In [15]:
def mk_quad(a,b,c): return partial(quad, a,b,c)

In [16]:
f = mk_quad(3,2,1)
f(1.5)

10.75

In [ ]:
plot_function(f)

In [18]:
def noise(x, scale): return np.random.normal(scale=scale, size=x.shape)
def add_noise(x, mult, add): return x * (1+noise(x,mult)) + noise(x,add)

In [21]:
np.random.seed(42)

x = torch.linspace(-2, 2, steps=20)[:,None]
y = add_noise(f(x), 0.3, 1.5)

C:\Users\VeeneetKumar\AppData\Local\Temp\ipykernel_8896\1958766108.py:2: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  def add_noise(x, mult, add): return x * (1+noise(x,mult)) + noise(x,add)


In [ ]:
plt.scatter(x,y)

In [ ]:
@interact(a=1.5, b=1.5, c=1.5)
def plot_quad(a, b, c):
    plt.scatter(x,y)
    plot_function(mk_quad(a,b,c), ylim=(-3,12))

In [24]:
def mse(preds, acts): return ((preds - acts)**2).mean()

In [ ]:
@interact(a=1.5, b=1.5, c=1.5)
def plot_quad(a,b,c):
    f = mk_quad(a,b,c)
    plt.scatter(x,y)
    loss=mse(f(x), y)
    plot_function(f, ylim=(-3,12), title=f"MSE: {loss:.2f}")

In [26]:
def quad_mse(params):
    f = mk_quad(*params)
    return mse(f(x), y)

In [27]:
quad_mse([1.5, 1.5, 1.5])

tensor(5.8336, dtype=torch.float64)

In [28]:
# Rank 1 tensor
abc = torch.tensor([1.5, 1.5, 1.5])
abc.requires_grad_()

tensor([1.5000, 1.5000, 1.5000], requires_grad=True)

In [29]:
loss = quad_mse(abc)
loss

tensor(5.8336, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [30]:
loss.backward()

In [31]:
abc.grad

tensor([-5.1419,  2.8472, -1.1009])

In [33]:
with torch.no_grad():
    abc -= abc.grad * 0.01
    loss = quad_mse(abc)

print(f'loss={loss:.2f}')

loss=3.51


In [34]:
# Automate it
for i in range(5):
    loss = quad_mse(abc)
    loss.backward()
    with torch.no_grad():
        abc -= abc.grad * 0.01
    print(f'step={i}; loss={loss:.2f}')

step=0; loss=3.51
step=1; loss=3.41
step=2; loss=3.31
step=3; loss=3.20
step=4; loss=3.09


In [35]:
abc

tensor([2.2987, 0.7915, 1.5252], requires_grad=True)

In [36]:
def rectified_linear(m, b, x):
    y = m*x+b
    return torch.clip(y, 0.)

In [ ]:
plot_function(partial(rectified_linear,1,1))

In [ ]:
@interact(m=1.5, b=1.5)
def plot_relu(m, b):
    plot_function(partial(rectified_linear, m, b), ylim=(-1,4))

In [39]:
def double_relu(m1, b1, m2, b2, x):
    return rectified_linear(m1, b1, x) + rectified_linear(m2, b2, x)

In [ ]:
@interact(m1=1.5, b1=1.5, m2=1.5, b2=1.5)
def plot_double_relu(m1, b1, m2, b2):
    plot_function(partial(double_relu, m1, b1, m2, b2), ylim=(-1,6))